This is playing with pytorch framework for EHR modeling. In general, a patient's health record can be represented as a sequence of visits. Each visit has certain features, and can be represented as a list of medical codes.

For simplicity, we are starting with the data structure that a patient's health record is a list of list, following the line of work from Jimeng Sun's lab. We will use codes from Ed Choi to manipulate the data. 

The core model is an GRU.

# todos:
* dropout
* L1/L2 regularization
* validation and AUC

In [1]:
%matplotlib inline
from __future__ import print_function, division
from io import open
import string
import re
import random
import sklearn 
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.utils.data as Data

use_cuda = torch.cuda.is_available()

import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

In [2]:
use_cuda

False

In [3]:
# prepare data: load the input file containing list of list of list, and corresponding label file
# and output the splitted training, valid and Test sets

def data_load_split_VT(seqFile = 'pytorch_ehr-master/pytorch_ehr-master/data/cerner/hospital_data/h143.visits', labelFile = 'pytorch_ehr-master/pytorch_ehr-master/data/cerner/hospital_data/h143.labels' , test_r=0.2 , valid_r=0.1):

    set_x = pickle.load(open(seqFile, 'rb'), encoding='bytes')
    set_y = pickle.load(open(labelFile, 'rb'),encoding='bytes')
    merged_set = [[set_y[i],set_x[i]] for i in range(len(set_x))] # merge the two lists

    # set random seed
    random.seed( 3 )
    
    dataSize = len(merged_set)
    nTest = int(test_r * dataSize)
    nValid = int(valid_r * dataSize)
    
    random.shuffle(merged_set)

    test_set = merged_set[:nTest]
    valid_set = merged_set[nTest:nTest+nValid]
    train_set = merged_set[nTest+nValid:]

    return train_set, valid_set, test_set

In [4]:
train_sl , valid_sl , test_sl = data_load_split_VT()

In [5]:
len(test_sl)

8545

In [6]:
class EHR_RETAIN(nn.Module):
    def __init__(self, input_size, hidden_size,embed_dim, n_layers=1,dropout_r=0.1):
        super(EHR_RETAIN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cuda_flag = torch.cuda.is_available()
        self.release = False # if set to true, then we return all values in computing

#        self.embedding = nn.Embedding(input_size, hidden_size)
        self.embedBag = nn.EmbeddingBag(input_size, self.embed_dim,mode= 'sum')
#        self.gru = nn.GRU(self.embed_dim, hidden_size, dropout= dropout_r )
#        self.out = nn.Linear(self.hidden_size,1)
#        self.sigmoid = nn.Sigmoid()
        
        self.RNN1 = nn.GRU(embed_dim,hidden_size,1,batch_first=True,bidirectional=True,dropout= dropout_r) # for alpha
        self.RNN2 = nn.GRU(embed_dim,hidden_size,1,batch_first=True,bidirectional=True,dropout= dropout_r) # for Beta
        self.RNN3 = nn.GRU(embed_dim,hidden_size,1,batch_first=True,bidirectional=True,dropout= dropout_r) # for attention
        self.wa = nn.Linear(hidden_size*2,1,bias=False)
        self.Wb = nn.Linear(hidden_size*2,hidden_size,bias=False)
        self.W_out = nn.Linear(hidden_size,1,bias=False)###replaced num_classes with 1
        self.wa2 = nn.Linear(hidden_size*2, 1)
        self.w_lever = nn.Linear(hidden_size*2, 2)


    ##def __init__(self, input_size, hidden_size, num_classes, cuda_flag=False):
        ##super(RETAIN,self).__init__()
        ##emb = nn.Embedding(1400,input_size)
        ##self.emb = emb.weight
        ##emb_trg = nn.Embedding(num_classes,num_classes)
        ##emb_trg.weight.data.copy_(torch.from_numpy(np.eye(num_classes)))
        ##self.emb_trg = emb_trg


      
    def EmbedPatient_MB(self, seq_mini_batch): # x is a ehr_seq_tensor
        
        lp= len(max(seq_mini_batch, key=lambda xmb: len(xmb[1]))[1]) # max number of visitgs within mb ??? verify again
        tb= torch.FloatTensor(len(seq_mini_batch),lp,self.embed_dim) 
        lbt1= torch.FloatTensor(len(seq_mini_batch),1)

        for pt in range(len(seq_mini_batch)):
            
            lbt ,pt_visits =seq_mini_batch[pt]
            lbt1[pt] = torch.FloatTensor([[float(lbt)]])
            ml=(len(max(pt_visits, key=len))) ## getting the visit with max no. of codes ##the max number of visits for pts within the minibatch
            txs= torch.LongTensor(len(pt_visits),ml)
            
            b=0
            for i in pt_visits:
                pd=(0, ml-len(i))
                txs[b] = F.pad(torch.from_numpy(np.asarray(i)).view(1,-1),pd,"constant", 0).data
                b=b+1
            
            if use_cuda:
                txs=txs.cuda()
                
            emb_bp= self.embedBag(Variable(txs)) ### embed will be num_of_visits*max_num_codes*embed_dim 
            #### the embed Bag dim will be num_of_visits*embed_dim
            
            zp= nn.ZeroPad2d((0,0,0,(lp-len(pt_visits))))
            xzp= zp(emb_bp)
            tb[pt]=xzp.data

        #tb= tb.permute(1, 0, 2) ### as my final input need to be seq_len x batch_size x input_size
        #commented the above as they use batch first
        emb_m=Variable(tb)
        label_tensor = Variable(lbt1)

        if use_cuda:
                label_tensor = label_tensor.cuda()
                emb_m = emb_m.cuda()
                
        return emb_m , label_tensor
    

 
    def forward(self, inputs):#,hidden):
        # get embedding using self.emb
        #b,seq,features = inputs.size()
        #embedded = torch.mm(inputs.view(-1,features),self.emb).view(b,seq,-1)
        #if self.release:
        #    self.embedded = embedded
            x_in , lt = self.EmbedPatient_MB(inputs)
          #  print (lt,x_in)
            b,seq,features = x_in.size()
          #  print ("b,seq,features",b,seq,features)
            # get alpha coefficients
            outputs1 = self.RNN1(x_in) # [b x seq x 128*2] , the hidden on 2*b*128
           # print('outputs1', outputs1)
            E = self.wa(outputs1[0].contiguous().view(-1, self.hidden_size*2)) # [b*seq x 1]
            #print('E', E)
            alpha = F.softmax(E.view(b,seq),1) # [b x seq]
            if self.release:
                self.alpha = alpha
            #print('alpha', alpha)
            # get beta coefficients
            outputs2 = self.RNN2(x_in) # [b x seq x 128] actually is the same as RNN1 [b x seq x 128*2] , the hidden on 2*b*128
            #print('outputs2', outputs2)
            outputs2 = self.Wb(outputs2[0].contiguous().view(-1,self.hidden_size*2)) # [b*seq x hid]
            #print('outputs2 after wb', outputs2)

            Beta = torch.tanh(outputs2).view(b, seq, self.hidden_size) # [b x seq x 128]
            #print('Beta', Beta)

            if self.release:
                self.Beta = Beta
            
            outputs_calc = self.compute(x_in, Beta, alpha) # [b, num_classes]
            #print('outputs_calc', outputs_calc)
            outputs_calc = F.softmax(outputs_calc,1)
            #print('outputs_calc after softmax', outputs_calc)
            
            # get outputs obtained by copying mechanism
            outputs3 = self.RNN3(x_in) # [b, seq, 128]
            #print('outputs3', outputs3)
            E = self.wa2(outputs3[0].contiguous().view(-1, self.hidden_size*2)) # [b*seq x 1]
            #print('E after out3', E)
            alpha2 = F.softmax(E.view(b,seq),1).unsqueeze(2) # [b, seq, 1]
            #print('alpha2', alpha2)
            #alpha2 = alpha2[:,1:,:] # [b,seq-1,1]
            #print('alpha2 after slicing', alpha2)
            if self.release:
                self.alpha2 = alpha2

            # targets: list of list
            targets = lt
            #targets = Variable(torch.LongTensor(targets)[:,:-1],requires_grad=False)
            if self.cuda_flag:
                targets = targets.cuda()
            #inputs2 = targets #self.emb_trg(targets) # [b, seq-1, num_classes]
            #print('targets inputs2', inputs2)
            #outputs_copy = (inputs2*alpha2).sum(1) # [b, num_classes] .expand_as(inputs2)
            #outputs_copy = F.softmax(outputs_copy,1)
            #print('output_copy',outputs_copy)

            # for inputs, we apply another contribution score that helps add up to output
            # this way, we can obtain a vector that takes in all of the inputs

            lever = outputs3[0].sum(1) # [b, 128]
            #print('lever [b, 128]',lever)
            lever = F.softmax(self.w_lever(lever),1) # [b,2]
            #print('lever after w_lever and softmax [b, 2]',lever)
            if self.release:
                self.lever = lever
                self.outputs_calc = outputs_calc
                self.outputs_copy = outputs_copy

            #outputs = outputs_calc*lever[:,0:1].expand_as(outputs_calc) + outputs_copy*lever[:,1:2].expand_as(outputs_copy) # [b, num_classes]
            outputs = outputs_calc*lever[:,0:1].expand_as(outputs_calc)
            
            #print('final output',outputs)
            return outputs , targets

    # multiply to inputs
    
    def compute(self, embedded, Beta, alpha):
            b,seq,_ = embedded.size()
            outputs = (embedded*Beta)*alpha.unsqueeze(2).expand(b,seq,self.hidden_size)
            outputs = outputs.sum(1) # [b x hidden]
            return self.W_out(outputs) # [b x num_classes]
        



    
'''    def forward(self, input, hidden):
        
        x_in , lt = self.EmbedPatient_MB(input)
        
        for i in range(self.n_layers):
                output, hidden = self.gru(x_in, hidden) # input (seq_len, batch, input_size) need to check torch.nn.utils.rnn.pack_padded_sequence() 
                                                          #h_0 (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. Defaults to zero if not provided.

        output = self.sigmoid(self.out(output[0]))
        return output, lt
'''


'    def forward(self, input, hidden):\n        \n        x_in , lt = self.EmbedPatient_MB(input)\n        \n        for i in range(self.n_layers):\n                output, hidden = self.gru(x_in, hidden) # input (seq_len, batch, input_size) need to check torch.nn.utils.rnn.pack_padded_sequence() \n                                                          #h_0 (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. Defaults to zero if not provided.\n\n        output = self.sigmoid(self.out(output[0]))\n        return output, lt\n'

In [7]:

model = EHR_RETAIN(input_size=20000, hidden_size=128 ,embed_dim=128, dropout_r=0)

if use_cuda:
    model = model.cuda()

In [8]:
def train (mini_batch, criterion, optimizer):  
    
    #hidden = model.initHidden()
    model.zero_grad()
    output , label_tensor = model(mini_batch)#,hidden)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.data[0]

In [9]:
def variableFromEHRSeq(ehr_seq):
    # ehr_seq is a list of list
    result = []
    if use_cuda:
        for i in range(len(ehr_seq)):
            result.append( Variable(torch.LongTensor([int(v) for v in ehr_seq[i]])).cuda() )
    # if use_cuda:
    #     return result.cuda()
    else:
        for i in range(len(ehr_seq)):
            result.append( Variable(torch.LongTensor([int(v) for v in ehr_seq[i]])) )

    return result

In [10]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)





In [11]:
def run_model_train(dataset,batch_size,learning_rate = 0.0001 ):
    
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    #optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=0)
    optimizer = optim.Adam(model.parameters(), weight_decay=0)
    #optimizer = optim.RMSprop (model.parameters())

    dataset.sort(key=lambda pt:len(pt[1])) 
   
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    #print('number of Batches',n_batches)
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
             #Print iter number, loss, name and guess
            if iter % print_every == 0:
                   print('%d %d%% (%s) %.4f ' % ( iter, iter/ n_batches * 100, timeSince(start), loss))

            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses


In [12]:
def calculate_auc(test_model, dataset, batch_size=200):

    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []

    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = model(batch)
            y_hat.append(output.cpu().data.numpy()[0][0])
            labelVec.append(label_t.cpu().data.numpy()[0][0])
    #print (labelVec, y_hat)
    auc = roc_auc_score(labelVec, y_hat)
    return auc

In [13]:
epochs=20
batch_size=200
current_loss_l=[]
all_losses_l=[]

for ep in range(epochs):
    #print ("Epoch ", ep )
    current_loss_la,all_losses_la = run_model_train(train_sl,batch_size)
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc)
    current_loss_l.append(current_loss_la)
    all_losses_l.append (all_losses_la)

10 6% (0m 5s) 0.8907 
20 13% (0m 8s) 0.3418 
30 20% (0m 12s) 0.3307 
40 26% (0m 14s) 0.5247 
50 33% (0m 16s) 0.4654 
60 40% (0m 19s) 0.3925 
70 46% (0m 32s) 0.3190 
80 53% (0m 36s) 1.6318 
90 60% (0m 40s) 1.2937 
100 66% (0m 47s) 0.3815 
110 73% (0m 54s) 0.3937 
120 80% (1m 2s) 0.3774 
130 86% (1m 8s) 0.8249 
140 93% (1m 13s) 0.3453 
150 100% (1m 16s) 0.4897 
Epoch  0  Train_auc : 0.705882352941  , Valid_auc :  0.564705882353  ,& Test_auc :  0.724358974359
10 6% (0m 2s) 0.4544 
20 13% (0m 11s) 0.4663 
30 20% (0m 15s) 1.2552 
40 26% (0m 20s) 0.3725 
50 33% (0m 26s) 0.3533 
60 40% (0m 28s) 0.3747 
70 46% (0m 34s) 0.5775 
80 53% (0m 36s) 0.3851 
90 60% (0m 39s) 0.4254 
100 66% (0m 43s) 1.1858 
110 73% (0m 47s) 0.4770 
120 80% (0m 51s) 3.8466 
130 86% (0m 55s) 1.7707 
140 93% (0m 57s) 0.4735 
150 100% (1m 9s) 5.7559 
Epoch  1  Train_auc : 0.669615214507  , Valid_auc :  0.6  ,& Test_auc :  0.75641025641
10 6% (0m 5s) 1.6856 
20 13% (0m 9s) 0.3540 
30 20% (0m 12s) 0.2998 
40 26% (0m 14s) 0.4

KeyboardInterrupt: 

In [14]:
all_losses_l

[[1.580540269613266,
  0.6151448905467987,
  0.5401335477828979,
  0.6278171360492706,
  0.6401678860187531,
  0.3816168010234833,
  0.39800177812576293,
  0.3970666706562042,
  0.5435400366783142,
  0.4139790594577789,
  0.49660996794700624,
  0.5397546768188477,
  1.2314470291137696,
  4.3527842402458194,
  1.3446409463882447,
  0.6595134794712066,
  1.5113649427890778,
  0.8701830089092255,
  0.8862375855445862,
  1.579005056619644,
  1.933153223991394,
  0.4697125911712646,
  0.9309005022048951,
  1.4198203682899475,
  1.7455278217792511,
  1.124794602394104,
  1.342689448595047,
  0.7003886222839355,
  0.6690422296524048,
  0.6423026204109192],
 [0.44635125398635866,
  0.6875577390193939,
  1.2218059182167054,
  0.4167063057422638,
  0.6810052275657654,
  0.5981203734874725,
  0.5208072662353516,
  0.619415020942688,
  0.5455323576927185,
  1.3128721117973328,
  0.4281072080135345,
  0.40259002447128295,
  1.0054753065109252,
  0.7001903891563416,
  0.5961590051651001,
  0.3993136

In [15]:
for x_ep in all_losses_l:
    y_ep = np.mean(x_ep)
    print (y_ep)

1.01959603469
0.808482508858
0.875690043668
0.684696993132
0.636889290015
